# Simulating low-light product inspection

This notebook shows how to systematically simulate a change in lighting conditions
(i.e. a shift in data distribution), and the effect such a change has on model
predictions. The notebook is using the Intel® Geti™ SDK to interact with an
Intel® Geti™ server.

**Problem description**: For any given project, a data scientist working on it may
want to quantify the effect of image distortions on model performance. For example in
medical imaging, there is always a certain noise background in the images depending on
patient anatomy and radiation dose.

In this scenario the customer has set up an anomaly segmentation model to detect
fabrication defects on discrete transistors mounted on a printed circuit board. For
various reasons, the customer wants to reduce both the light intensity and exposure
time in the inspection line. Let's assume the product they are inspecting is sensitive
to light so they want to achieve the minimal exposure to light possible, but they
still want to detect fabrication defects. How will this affect their anomaly
segmentation model? Of course they will do real-world tests before implementing
any changes, but is it possible to simulate such a shift in data distribution in
advance? *With Intel® Geti™, it is*.

**Project type**: Anomaly segmentation

**Project name**: Transistor anomaly segmentation


### Step 1: Connect to the Intel® Geti™ server

In [ ]:
from dotenv import dotenv_values

from geti_sdk import Geti

env_variables = dotenv_values(dotenv_path="../.env")

# The Geti object establishes the connection to the Intel® Geti™ platform
geti = Geti(
    host=env_variables.get("HOST"),
    username=env_variables.get("USERNAME"),
    password=env_variables.get("PASSWORD"),
)

### Step 2: Get project
The utility function `ensure_trained_anomaly_project` checks that the project required
for this notebook is found on the server. If it is not found, the function will create
it. To do so, it may have to download the dataset, which could take some time.

In [ ]:
from geti_sdk.demos import ensure_trained_anomaly_project

PROJECT_NAME = "Transistor anomaly classification"
project = ensure_trained_anomaly_project(geti=geti, project_name=PROJECT_NAME)

print(project.summary)

### Step 3: Get images and annotations
Set up ImageClient for the project, get image metadata

In [ ]:
from geti_sdk.rest_clients import ImageClient

image_client = ImageClient(
    session=geti.session, workspace_id=geti.workspace_id, project=project
)

images = image_client.get_all_images()
print(f"Project '{project.name}' contains {len(images)} images")

image_1 = images[0]

Set up AnnotationClient for the project, get annotations for first image in the project and inspect it.

In [ ]:
from geti_sdk.rest_clients import AnnotationClient
from geti_sdk.utils import show_image_with_annotation_scene

annotation_client = AnnotationClient(
    session=geti.session, workspace_id=geti.workspace_id, project=project
)

annotation_1 = annotation_client.get_annotation(image_1)

# Inspect annotation for image 1
image_1.get_data(geti.session)
show_image_with_annotation_scene(image_1, annotation_1, show_in_notebook=True)

Get and inspect anomalous image with annotation

In [ ]:
image_2 = images[1]

annotation_2 = annotation_client.get_annotation(image_2)

# Inspect annotation for image 2
image_2.get_data(geti.session)
show_image_with_annotation_scene(image_2, annotation_2, show_in_notebook=True)

### Step 4: Get prediction for anomalous image
Set up prediction client, fetch prediction for image

In [ ]:
from geti_sdk.rest_clients import PredictionClient

prediction_client = PredictionClient(
    session=geti.session, workspace_id=geti.workspace_id, project=project
)

prediction = prediction_client.get_image_prediction(image_2)

show_image_with_annotation_scene(image_2, prediction, show_in_notebook=True)

### Step 5: Simulate low light conditions
To simulate the reduced light intensity, we decrease the overall brightness and add shot noise to the image

In [ ]:
from utils import display_image_in_notebook, simulate_low_light_image

# Reduce brightness and add shot noise to
# simulate low light intensity and short exposure time
new_image_with_noise = simulate_low_light_image(image_2, reduction_factor=0.75)

display_image_in_notebook(new_image_with_noise)

### Step 6: Get prediction for the modified image

In [ ]:
sc_noisy_image, noisy_prediction = geti.upload_and_predict_image(
    image=new_image_with_noise, project_name=PROJECT_NAME, visualise_output=False
)
show_image_with_annotation_scene(
    new_image_with_noise, noisy_prediction, show_in_notebook=True
)

### Step 7: Simulate a range of different light levels
Change the lighting reduction factor from very strong reduction (`reduction_factor=0.1`) to weak reduction (`reduction_factor=0.8`). To inspect the resulting images, it is easiest to open the Geti UI in your browser and have a look at the dataset for the project. The modified images should show up at the bottom of the image list in the project.

In [ ]:
import numpy as np

start_factor = 0.1
stop_factor = 0.8
step = 0.1

for alpha in np.arange(start_factor, stop_factor, step):
    new_image_with_noise = simulate_low_light_image(image_2, reduction_factor=alpha)
    image, prediction = geti.upload_and_predict_image(
        image=new_image_with_noise, project_name=PROJECT_NAME, visualise_output=False
    )
    print(
        f"Light reduction factor: {alpha:.1f}. Model prediction: {prediction.annotations[0].labels[0].name}"
    )

## Model re-training for low light conditions
Ok, so now we know that the *existing* model can still find anomalies, even in low light conditions. But, of course this is not a fair comparison since the low light images are not part of the training set for that model. Could we simulate training a completely new model on a low-light dataset? Yes we can!

### Step 8: Create a new project
Create a new project dedicated to images with a certain lighting reduction factor

In [ ]:
from geti_sdk.rest_clients import ProjectClient

project_client = ProjectClient(session=geti.session, workspace_id=geti.workspace_id)

LIGHT_REDUCTION_FACTOR = 0.5

new_project = project_client.get_or_create_project(
    project_name=PROJECT_NAME + f" light reduction factor {LIGHT_REDUCTION_FACTOR:.1f}",
    project_type="anomaly_classification",
    labels=[[]],
)

### Step 9: Modify all images data and annotations

##### Loop over the existing images and:
  1. Create new image with simulated low light conditions
  2. Get existing annotation for image
  3. Apply existing annotation to simulated image

In [ ]:
new_image_client = ImageClient(
    session=geti.session, workspace_id=geti.workspace_id, project=new_project
)
new_annotation_client = AnnotationClient(
    session=geti.session, workspace_id=geti.workspace_id, project=new_project
)

print(
    f"Converting and uploading images and annotations to project '{new_project.name}'..."
)
for ii, image in enumerate(images):
    annotation = annotation_client.get_annotation(image)
    numpy_image = image.get_data(geti.session)
    new_image = simulate_low_light_image(
        numpy_image, reduction_factor=LIGHT_REDUCTION_FACTOR
    )
    new_sc_image = new_image_client.upload_image(new_image)
    new_annotation = annotation.map_labels(labels=new_project.get_all_labels())
    new_annotation_client.upload_annotation(
        annotation_scene=new_annotation, media_item=new_sc_image
    )
    if (ii + 1) % 25 == 0:
        print(
            f"{ii+1} images and annotations converted and uploaded successfully. Processing..."
        )

print("Conversion complete")

### Step 10: Start training job and monitor progress

In [ ]:
from geti_sdk.rest_clients import TrainingClient

training_client = TrainingClient(
    session=geti.session, workspace_id=geti.workspace_id, project=new_project
)

job = training_client.train_task(task=0)

training_client.monitor_jobs([job])

### Step 11: Comparing the two models
Now that we have a trained model for both projects, we can compare the performance between the two. 

In [ ]:
from geti_sdk.rest_clients import ModelClient

model_client = ModelClient(
    session=geti.session, workspace_id=geti.workspace_id, project=project
)
new_model_client = ModelClient(
    session=geti.session, workspace_id=geti.workspace_id, project=new_project
)

model = model_client.get_all_active_models()[0]
new_model = new_model_client.get_all_active_models()[0]

print(
    f"Performance of the model for the unmodified, original project: {model.performance.score}"
)
print(
    f"Performance of the model for new project with light reduction factor of {LIGHT_REDUCTION_FACTOR}: {new_model.performance.score}"
)

### Finally, cleaning up

In [ ]:
# Cleaning up the first project by removing the uploaded images that don't have annotations

images_to_delete = []
for image in image_client.get_all_images():
    if annotation_client.get_annotation(image) is None:
        images_to_delete.append(image)
image_client.delete_images(images_to_delete)

In [ ]:
# Cleaning up the second project by removing it from the server
project_client.delete_project(project=new_project)